In [ ]:
# Surprise 설치
!pip install scikit-surprise
# '무비 렌즈'라는 공개된 오픈 데이터셋에 기초해서 만들어진 추천시스템

     |████████████████████████████████| 11.8 MB 5.0 MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1619432 sha256=dc072256d636205c10094d52fe739d28a5636752a33ba52e5bfe53ebfabcb2fc
  Stored in directory: /root/.cache/pip/wheels/76/44/74/b498c42be47b2406bd27994e16c5188e337c657025ab400c1c
Successfully built scikit-surprise


In [ ]:
import pandas as pd
import os
import surprise
import matplotlib.pyplot as plt

In [ ]:
Ratings=pd.read_csv("./drive/MyDrive/data-files/Ratings.csv")
Ratings.to_csv('Ratings_surprise.csv', index=False, header=False) # Suprise 무비렌즈의 포맷은 index와 header가 없는 상태
print(Ratings.shape)
Ratings.head(1)

(8082, 4)


,user,item,rating,timestamp
0,8,Life in Loops (A Megacities RMX),7.5,18


In [ ]:
# null 체크
Ratings.isnull().sum()

user         0
item         0
rating       0
timestamp    0
dtype: int64

In [ ]:
# Surprise가 인지하고 있는 무비렌즈 활용
data = surprise.Dataset.load_builtin('ml-100k')

Dataset ml-100k could not be found. Do you want to download it? [Y/n] y
Trying to download dataset from http://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to /root/.surprise_data/ml-100k


In [ ]:
df = pd.DataFrame(data.raw_ratings, columns=["user", "item", "rating", "timestamp"])
del df["timestamp"]
df.head(10)

,user,item,rating
0,196,242,3.0
1,186,302,3.0
2,22,377,1.0
3,244,51,2.0
4,166,346,1.0
5,298,474,4.0
6,115,265,2.0
7,253,465,5.0
8,305,451,3.0
9,6,86,3.0


In [ ]:
df_table = df.set_index(["user", "item"]).unstack()
df_table.shape

(943, 1682)

In [ ]:
df_table.iloc[212:222, 808:817].fillna("")

rating                                
item    211 212 213 214 215 216 217 218 219
user                                       
290       3                   4       2    
291           4       4   4           4   4
292                   3                    
293       4       3       4   4   3   2    
294                                        
295               5       5   5   4   5    
296       4                                
297       4       3       2   4       3    
298       5       3       5                
299       4   4   5           5

In [ ]:
# 추천성능 평가기준
import numpy as np
from surprise.model_selection import KFold

bsl_options = {
    'method': 'als',
    'n_epochs': 5,
    'reg_u': 12,
    'reg_i': 5}
algo = surprise.BaselineOnly(bsl_options)

np.random.seed(0)
acc = np.zeros(3)
cv = KFold(3)
for i, (trainset, testset) in enumerate(cv.split(data)):
    algo.fit(trainset)
    predictions = algo.test(testset)
    acc[i] = surprise.accuracy.rmse(predictions, verbose=True)
acc.mean()

Estimating biases using als...
RMSE: 0.9453
Estimating biases using als...
RMSE: 0.9377
Estimating biases using als...
RMSE: 0.9500


0.9443304984013942

In [ ]:
# 평균제곱차이 유사도
sim_options = {'name': 'msd'}
algo = surprise.KNNBasic(sim_options=sim_options)
cross_validate(algo, data)["test_mae"].mean()

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.


0.773120723615596

In [ ]:
# 코사인 유사도 측정
# 두 벡터 간의 코사인 각도를 이용하여 구할 수 있는 두 벡터의 유사도
sim_options = {'name': 'cosine'}
algo = surprise.KNNBasic(sim_options=sim_options)
cross_validate(algo, data)["test_mae"].mean()

Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.


0.8043456067429883

In [ ]:
# 피어슨 유사도
# 두 벡터가 주어졌을 때의 상관관계를 계산
sim_options = {'name': 'pearson'}
algo = surprise.KNNBasic(sim_options=sim_options)
cross_validate(algo, data)["test_mae"].mean()

Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.


0.8029650520748135

In [ ]:
# 피어슨-베이스라인 유사도
# 선형회귀 모델과 비슷
sim_options = {'name': 'pearson_baseline'}
algo = surprise.KNNBasic(sim_options=sim_options)
cross_validate(algo, data)["test_mae"].mean()

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


0.7906769982717502

In [ ]:
# KNN 가중치 예측 방법
# 새로운 데이터가 주어졌을 때, 기존 데이터 중 가장 가까운 k개의 데이터를 정보로 새로운 데이터를 예측하는 방법론
# KNNWithMeans
sim_options = {'name': 'pearson_baseline'}
algo = surprise.KNNWithMeans(sim_options=sim_options)
cross_validate(algo, data)["test_mae"].mean()

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


0.7297849328480457

In [ ]:
# KNNBaseline (기본적인 협업필터링(Collaborative Filtering) 알고리즘, 추가적으로 baseline을 더함)
sim_options = {'name': 'pearson_baseline'}
algo = surprise.KNNBaseline(sim_options=sim_options)
cross_validate(algo, data)["test_mae"].mean()

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


0.7213491086974948

In [ ]:
# SVD (특이값 분해 Singular Value Decomposition)

algo = surprise.SVD(n_factors=100)
cross_validate(algo, data)["test_mae"].mean()

0.7384746832340271

In [ ]:
# NMF(Non-negative matrix factorization)
# 음수 미포함 행렬 분해(Non-negative Matrix Factorization, NMF)는 음수를 포함하지 않는 행렬 X를 음수를 포함하지 않는 행렬 W와 H의 곱으로 분해하는 알고리즘

algo = surprise.NMF(n_factors=100)
cross_validate(algo, data)["test_mae"].mean()

0.8375203469881669